In [1]:
import numpy as np
import seaborn as sns
import sympy as sp
import pandas as pd
import scipy

In [2]:
dados = {'d': [.025, .05, .025, .05], 
         'l': [2.7, 2.8, 1.5, 1.35], 
         'qtd_feixe': [12, 25, 8, 12],}

#dados = {'d': [.025, .05, .025, .05], 
#         'l': [2.7, 2.8, 1.5, 1.35], 
#         'qtd_feixe': [11, 22, 6, 11],} #caso de contagem ideal

In [3]:
# Interferencia
df = pd.DataFrame(dados)

In [4]:
df

,d,l,qtd_feixe
0,0.025,2.70,12
1,0.050,2.80,25
2,0.025,1.50,8
3,0.050,1.35,12


In [5]:
df['sep_entre_2_franjas'] = df['l'] / df['qtd_feixe']

In [6]:
df['erro_da_medida_entre_franjas'] = (0.05/df['qtd_feixe'])#/df['sep_entre_2_franjas']

sabendo a eq de interferencia para angulos muito pequenos

$$\Delta y = \frac{D * \lambda}{d}$$


$$\lambda = \frac{\Delta y * d}{D}$$


In [7]:
df['lambda'] = df['d'] * df['sep_entre_2_franjas'] / 96.5 * 1e7 # cm p/ nm
df['lambda']

0    582.901554
1    580.310881
2    485.751295
3    582.901554
Name: lambda, dtype: float64

In [8]:
df['valor_bom_p_sep_das_franjas'] = 632.8 * 96.5 / (1e7 * df['d']) # cm p/ nm
df['valor_bom_p_sep_das_franjas']

0    0.244261
1    0.122130
2    0.244261
3    0.122130
Name: valor_bom_p_sep_das_franjas, dtype: float64

In [9]:
df[['valor_bom_p_sep_das_franjas' , 'sep_entre_2_franjas']]

,valor_bom_p_sep_das_franjas,sep_entre_2_franjas
0,0.244261,0.2250
1,0.122130,0.1120
2,0.244261,0.1875
3,0.122130,0.1125


In [10]:
df['possivel_qtd_ideal'] = 1/(df['valor_bom_p_sep_das_franjas'] / df['l'])

In [11]:
df[['possivel_qtd_ideal', 'qtd_feixe']]

,possivel_qtd_ideal,qtd_feixe
0,11.053759,12
1,22.926315,25
2,6.140977,8
3,11.053759,12


calculo do erro de D:
<br>
0.05 = erro da régua
<br>
96.5 = distância da fenda até o papel

In [12]:
df['erro_D'] = round(0.05/96.5, 4)

calculando as incertezas:
$\sigma_{\lambda} = \lambda \sqrt{\sigma_y^2 + \sigma_D^2}$

In [13]:
df[['erro_da_medida_entre_franjas', 'sep_entre_2_franjas']]

,erro_da_medida_entre_franjas,sep_entre_2_franjas
0,0.004167,0.2250
1,0.002000,0.1120
2,0.006250,0.1875
3,0.004167,0.1125


In [14]:
df['incertezas'] = df['lambda'] * np.sqrt((df['erro_da_medida_entre_franjas']/df['sep_entre_2_franjas'])**2 + df['erro_D']**2)

In [15]:
df['compatibilidade'] = (632.8 - df['lambda'])/df['incertezas'] # 632.8 nm é o valor referencia
df[['compatibilidade']].abs()

,compatibilidade
0,4.620908
1,5.063216
2,9.080706
3,2.311085


calculando incerteza associada a todas as medicoes

In [16]:
sigma = np.sqrt(1/(1/df['incertezas']**2).sum()).round(3)
sigma

6.477

In [17]:
valor = ((np.sqrt(1/(1/df['incertezas']**2).sum())/df['incertezas'])**2 * df['lambda']).sum()
valor

566.3498924773228

In [18]:
print(f'{valor:.3f} ± {sigma:.3f}')

566.350 ± 6.477


compatibilidade final:

In [19]:
round(abs((566.350 - 632.8)/6.477), 2)

10.26

In [20]:
dados = {'d': [0.002, 0.004, 0.008, 0.016], 
         'l': [25.2, 28.5, 29.5, 6.6], 
         'qtd_feixe': [8, 19, 40, 17]}

In [21]:
# difracao
df = pd.DataFrame(dados)
df

,d,l,qtd_feixe
0,0.002,25.2,8
1,0.004,28.5,19
2,0.008,29.5,40
3,0.016,6.6,17


$$a\sin{\theta} = m*\lambda$$

onde para angulos muito pequenos podemos:

$$\sin{\theta} = \tan{\theta} = \theta = y/D$$

onde y é a distancia entre o máximo central e a faixa escura mais próxima, então teremos:

$$a*y/D = m*\lambda$$

$$\lambda = a*y/D $$

In [22]:
df['sep_entre_2_franjas'] = df['l'] / df['qtd_feixe']
df['sep_entre_2_franjas']

0    3.150000
1    1.500000
2    0.737500
3    0.388235
Name: sep_entre_2_franjas, dtype: float64

In [23]:
df['erro_da_medida_entre_franjas'] = (0.05/df['qtd_feixe'])
df['erro_da_medida_entre_franjas']

0    0.006250
1    0.002632
2    0.001250
3    0.002941
Name: erro_da_medida_entre_franjas, dtype: float64

In [24]:
df

,d,l,qtd_feixe,sep_entre_2_franjas,erro_da_medida_entre_franjas
0,0.002,25.2,8,3.150000,0.006250
1,0.004,28.5,19,1.500000,0.002632
2,0.008,29.5,40,0.737500,0.001250
3,0.016,6.6,17,0.388235,0.002941


In [25]:
df['lambda'] = df['d'] * df['sep_entre_2_franjas'] / 96.5 * 1e7 # cm p/ nm

In [26]:
df

,d,l,qtd_feixe,sep_entre_2_franjas,erro_da_medida_entre_franjas,lambda
0,0.002,25.2,8,3.150000,0.006250,652.849741
1,0.004,28.5,19,1.500000,0.002632,621.761658
2,0.008,29.5,40,0.737500,0.001250,611.398964
3,0.016,6.6,17,0.388235,0.002941,643.706187


agora, dado que já sabemos o valor de lambda, podemos calcular o valor de d

In [27]:
df['lambda_real'] = 632.8

In [28]:
df['d_previsto'] = (df['lambda_real'] * 96.5 / df['sep_entre_2_franjas']) / 1e7
df['d_previsto']

0    0.001939
1    0.004071
2    0.008280
3    0.015729
Name: d_previsto, dtype: float64

único erro que teríamos nesse caso:

In [29]:
df['erro_D'] = 0.05/96.5

agora calculando a incerteza associada a essas medidas:

In [30]:
df[['sep_entre_2_franjas', 'erro_da_medida_entre_franjas', 'd']]

,sep_entre_2_franjas,erro_da_medida_entre_franjas,d
0,3.150000,0.006250,0.002
1,1.500000,0.002632,0.004
2,0.737500,0.001250,0.008
3,0.388235,0.002941,0.016


In [31]:
df['d_previsto_incerteza'] = df['d_previsto'] * np.sqrt(df['erro_D']**2 + ((df['erro_da_medida_entre_franjas']/df['sep_entre_2_franjas'])**2))
df['d_previsto_incerteza']

0    0.000004
1    0.000007
2    0.000015
3    0.000119
Name: d_previsto_incerteza, dtype: float64

In [32]:
(df['erro_da_medida_entre_franjas']/df['sep_entre_2_franjas'])**2

0    0.000004
1    0.000003
2    0.000003
3    0.000057
dtype: float64

In [33]:
df['erro_D']**2

0    2.684636e-07
1    2.684636e-07
2    2.684636e-07
3    2.684636e-07
Name: erro_D, dtype: float64

In [34]:
df['d_previsto'].round(4)

0    0.0019
1    0.0041
2    0.0083
3    0.0157
Name: d_previsto, dtype: float64

In [35]:
df['compatibilidade'] = (df['d'] - df['d_previsto'])
(df['compatibilidade']/df['d_previsto_incerteza']).abs()

0    15.450686
1     9.535704
2    19.081849
3     2.269693
dtype: float64